In [3]:
import weaviate
from weaviate.classes.config import Configure

client = weaviate.connect_to_local()

questions = client.collections.create(
    name="Transcripts",
    vectorizer_config=Configure.Vectorizer.text2vec_ollama(     # Configure the Ollama embedding integration
        api_endpoint="http://host.docker.internal:11434",       # Allow Weaviate from within a Docker container to contact your Ollama instance
        model="nomic-embed-text",                               # The model to use
    ),
    generative_config=Configure.Generative.ollama(              # Configure the Ollama generative integration
        api_endpoint="http://host.docker.internal:11434",       # Allow Weaviate from within a Docker container to contact your Ollama instance
        model="llama3.2",                                       # The model to use
    )
)

client.close()  # Free up resources

/home/shubh/.local/lib/python3.12/site-packages/weaviate/collections/classes/config.py:2082: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for cls_field in self.model_fields:


UnexpectedStatusCodeError: Collection may not have been created properly.! Unexpected status code: 422, with response body: {'error': [{'message': 'class name Transcripts already exists'}]}.

In [1]:
import weaviate

client = weaviate.connect_to_local()

collections = client.collections.list_all()  # Returns a list of collection names
print("✅ Collections in Weaviate:")
for name in collections:
    print("-", name)

client.close()


✅ Collections in Weaviate:
- Transcript


In [2]:
import os
import weaviate
from weaviate.classes.config import Configure

# Step 1: Connect to Weaviate
client = weaviate.connect_to_local()

# Step 2: Create a Collection (if not exists)
collection_name = "Transcript"

if collection_name not in client.collections.list_all():
    transcript_collection = client.collections.create(
        name=collection_name,
        vectorizer_config=Configure.Vectorizer.text2vec_ollama(
            api_endpoint="http://localhost:11434",  # Ollama inside Docker
            model="nomic-embed-text"
        )
    )
else:
    transcript_collection = client.collections.get(collection_name)

# Step 3: Load .txt files and store

for filename in os.listdir():
    if filename.endswith(".txt"):
        file_path = os.path.join(filename)
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()
        
        # Use the filename (without extension) as title or id
        title = os.path.splitext(filename)[0]

        # Insert into Weaviate
        transcript_collection.data.insert({
            "title": title,
            "text": text
        })

print("✅ All transcripts uploaded and embedded in Weaviate!")

client.close()


✅ All transcripts uploaded and embedded in Weaviate!


In [14]:
from weaviate.classes.config import Property, DataType

collection_name = "Transcript"

if collection_name not in client.collections.list_all():
    transcript_collection = client.collections.create(
        name=collection_name,
        vectorizer_config=Configure.Vectorizer.text2vec_ollama(
            api_endpoint="http://localhost:11434", 
            model="nomic-embed-text"
        ),
        properties=[
            Property(name="title", data_type=DataType.TEXT, vectorize=False),
            Property(name="text", data_type=DataType.TEXT, vectorize=True),
        ]
    )
    print("✅ Collection created:", collection_name)
else:
    transcript_collection = client.collections.get(collection_name)
    print("✅ Collection already exists:", collection_name)


# Step 3: Load .txt files and store

for filename in os.listdir():
    if filename.endswith(".txt"):
        file_path = os.path.join(filename)
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()
        
        # Use the filename (without extension) as title or id
        title = os.path.splitext(filename)[0]

        # Insert into Weaviate
        transcript_collection.data.insert({
            "title": title,
            "text": text
        })

print("✅ All transcripts uploaded and embedded in Weaviate!")


/home/shubh/.local/lib/python3.12/site-packages/weaviate/collections/classes/config.py:2082: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for cls_field in self.model_fields:
/home/shubh/.local/lib/python3.12/site-packages/weaviate/warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(
/tmp/ipykernel_130526/3133806314.py:6: ResourceWarning: unclosed <socket.socket fd=130, family=2, type=1, proto=6, laddr=('127.0.0.1', 51666), raddr=('127.0.0.1', 8080)>
  transcript_collection = client.collections.create(


✅ Collection created: Transcript


✅ All transcripts uploaded and embedded in Weaviate!


In [13]:
client = weaviate.connect_to_local()
client.collections.delete(name="Transcript")
print("✅ Transcript collection deleted successfully!")

✅ Transcript collection deleted successfully!


In [3]:
# Connect again (if not already)
import weaviate

client = weaviate.connect_to_local()
collection = client.collections.get("Transcript")

# Fetch all objects (limit = 5 for demo)
results = collection.query.fetch_objects(limit=5)

for obj in results.objects:
    print(obj)

client.close()


Object(uuid=_WeaviateUUIDInt('05c4dea1-c919-4128-b941-80e1550b609d'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'text': "[1 seconds][Agent] : Thank you so much for holding. I have my colleague Georgia on the line from our support team. Georgia, just to confirm, I have confirmed all points of ID. The contact information is up to date, should be able to assist you from here. We're ready.\n[12 seconds][Customer] : Thank you.\n[13 seconds][Agent] : Thank you so much. Hi Irene, my name is Georgia from Real Insurance. How are you today?\n[19 seconds][Customer] : Hello, Good. Thank you. How are you?\n[21 seconds][Agent] : I'm good, thank you. My colleagues confirmed all your details for you is that and that's including address, phone number and e-mail as well.\n[30 seconds][Customer] : Yes, yes, that's correct.\n[31 seconds][Agent] : Thank you so much. OK

In [6]:
client = weaviate.connect_to_local()
uuid = "9bf5f7a1-4c43-4215-ad81-9d3df4d365b6"
transcript_collection = client.collections.get("Transcript")
transcript_collection.data.delete_by_id(uuid)


/home/shubh/.local/lib/python3.12/site-packages/weaviate/warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(
/tmp/ipykernel_140129/2768645967.py:1: ResourceWarning: unclosed <socket.socket fd=81, family=2, type=1, proto=6, laddr=('127.0.0.1', 50190), raddr=('127.0.0.1', 8080)>
  client = weaviate.connect_to_local()


True

In [2]:
import weaviate
import os

client = weaviate.Client(
    url="http://localhost:8080",  # Local instance
    additional_headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY")  # Provide OpenAI key via env variable
    }
)

# Check if Weaviate is ready
print("✅ Connected:", client.is_ready())


RuntimeError: The grpc package installed is at version 1.65.5, but the generated code in v1/weaviate_pb2_grpc.py depends on grpcio>=1.66.2. Please upgrade your grpc module to grpcio>=1.66.2 or downgrade your generated code using grpcio-tools<=1.65.5.